# Problema das p-Medianas Capacitado com Custos Fixos

![imagem.png](imagem/imagem.png)

## Modelagem do Problema

#### Índices
- i = 1, ..., m locais que podem receber instalações
- j = 1, ..., n centros consumidores

#### Parâmetros
- $c_{ij}$ custo para atender o ponto $j$ a partir da instalação $i$;
- $p$ quantidade de instalações a serem abertas;
- $d_j$ demanda no centro consumidor $j$;
- $C_i$ capacidade da localidade $i$;
- $f_i$ custo fixo para abrir a instalação em $i$;

##### Variáveis de Decisão (todas binárias)
- $x_{ij}$ indica se a instalação $i$ atenderá o centro consumidor $j$;
- $y_i$ indica se uma instalação será aberta no local $i$;

#### Função Objetivo
Custo total da solução (transporte + fixo)
$$
\sum_{i=1}^{m}\sum_{j=1}^{n}c_{ij}x_{ij} + \sum_{i=1}^{m}f_i y_i
$$

#### Função Restrição
Todos os clientes devem ser atendidos
$$
\sum_{i=1}^{m}x_{ij}=1, \;\; \forall j=1,...,n
$$
Restrição de capacidade das instalações disponíveis
$$
\sum_{j=1}^{n}d_{j}x_{ij}\leq C_{i}y_{i}, \;\; \forall i=1,...,m
$$
Exatamente `p` instalações devem ser abertas
$$
\sum_{i=1}^{m}y_{i}=p,
$$

In [1]:
import gurobipy as gp

In [2]:
# Dados do problema
qtd_fabricas = 5   # Quantidades de fábricas
qtd_clientes = 10  # Quantidades de clientes
valor_p = 2        # Quantidades de fábricas que queremos abrir

# Custos para atender toda a demanda (origens X destinos)
vet_custos = [[684, 439, 441, 1078, 806, 557, 443, 356, 1265, 322],
              [441, 728, 822, 21, 440, 728, 1202, 1121, 934, 1077],
              [791, 576, 574, 1197, 899, 692, 385, 322, 1306, 306],
              [700, 974, 1059, 275, 675, 954, 1458, 1380, 1055, 1337],
              [217, 375, 483, 641, 324, 503, 558, 470, 859, 424]]

# Capacidade nas origens
vet_capacidades = [800, 1000, 1400, 1000, 1200]

# Custos fixos para abrir cada origem
vet_custos_fixos = [2200, 2500, 3300, 2300, 2900]

# Demanda nos destinos
vet_demandas = [130, 210, 100, 250, 170, 160, 230, 190, 150, 140]

In [5]:
# Rótulos dos índice
fabricas = [f'Fabrica_{i}' for i in range(1, qtd_fabricas+1)]

# Rótulos dos clientes
clientes = [f'Cliente_{j}' for j in range(1, qtd_clientes+1)]

In [13]:
# Dicionário de custos
custos = {(fabricas[i], clientes[j]):vet_custos[i][j] for i in range(qtd_fabricas) for j in range(qtd_clientes)}

# Dicionário de demandas dos clientes
demandas = {clientes[j]:vet_demandas[j] for j in range(qtd_clientes)}

# Dicionário de capacidade
capacidades = {fabricas[i]:vet_capacidades[i] for i in range(qtd_fabricas)}

# Dicionário de custos fixos
custos_fixos = {fabricas[i]:vet_custos_fixos[i] for i in range(qtd_fabricas)}

In [11]:
# Cria Modelo
m = gp.Model()

# Adicionar variáveis de decisão
x = m.addVars(fabricas, clientes, vtype=gp.GRB.BINARY)
y = m.addVars(fabricas, vtype=gp.GRB.BINARY)

In [12]:
# Função Objetivo
m.setObjective(
    x.prod(custos) + y.prod(custos_fixos),
    sense=gp.GRB.MINIMIZE
)

In [14]:
# Restrições que garatem que todo cliente vai ser atendido
c1 = m.addConstrs(
    x.sum('*', j) == 1 for j in clientes
)

# Restrições que abrem exatamente p fábricas 
c2 = m.addConstr(
    y.sum('*') == valor_p
)

# Restrições de capacidade
c3 = m.addConstrs(
    gp.quicksum(demandas[j]*x[i,j] for j in clientes) <= capacidades[i]*y[i]
    for i in fabricas
)

In [15]:
# Executa o model
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (linux64)
Optimize a model with 16 rows, 55 columns and 110 nonzeros
Model fingerprint: 0x085ab662
Variable types: 0 continuous, 55 integer (55 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e+01, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 12990.000000
Presolve time: 0.04s
Presolved: 16 rows, 55 columns, 110 nonzeros
Variable types: 0 continuous, 55 integer (55 binary)

Root relaxation: objective 8.793667e+03, 15 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 8793.66667    0    5 12990.0000 8793.66667  32.3%     -    0s
H    0     0                    10777.000000 8793.66667  18.4%     -    0s
H    0     0                    9581.0000000 8793.66667  8.22%     -    0s
     0     0 9581.00000   

In [23]:
# Relação de fábricas abetas,designações e capacidade
fab_abertas = [fab for fab in fabricas if round(y[fab].X) == 1]

for fab in fab_abertas:
    print(fab)
    for cli in clientes:
        if round(x[fab,cli].X) == 1:
            print(cli, ' ', end='')
    livre = c3[fab].Slack
    usado = capacidade[fab] - livre
    print('')
    print('Capacidade Nominal:', capacidade[fab])
    print('Capacidade Usada:', usado)
    print('Capacidade Livre:', livre)
    print('')

Fabrica_1
Cliente_3  Cliente_7  Cliente_8  Cliente_10  
Capacidade Nominal: 800
Capacidade Usada: 660.0
Capacidade Livre: 140.0

Fabrica_5
Cliente_1  Cliente_2  Cliente_4  Cliente_5  Cliente_6  Cliente_9  
Capacidade Nominal: 1200
Capacidade Usada: 1070.0
Capacidade Livre: 130.0

